In [28]:
import parselmouth
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.metrics import classification_report

from os import path, listdir
import glob

from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

In [29]:
df1 = pd.read_csv('Task2features.csv')

In [30]:
df2 = pd.read_csv('filename.csv', names = ['name'])
df2

,name
0,hw3_speech_files/cc_001_anxiety_910.77_May-twe...
1,hw3_speech_files/cc_001_anxiety_916.11_Eight-h...
2,hw3_speech_files/cc_001_anxiety_918.66_Eight-h...
3,hw3_speech_files/cc_001_anxiety_928.48_Four-th...
4,hw3_speech_files/cc_001_anxiety_934.73_Nine-th...
...,...
2319,hw3_speech_files/mm_001_shame_2878.13_six-hund...
2320,hw3_speech_files/mm_001_shame_2883.75_six-hund...
2321,hw3_speech_files/mm_001_shame_2889.69_ten-thou...
2322,hw3_speech_files/mm_001_shame_2895.38_ten-thou...


In [32]:
cols = [col for col in df1.columns]
cols[181]

'F0_sma_max'

In [33]:
df1 = df1.drop(cols[181], axis=1)

In [34]:
df1 = df1.drop(['name', 'class'], axis=1)

In [35]:
target = []
speaker = []
for index, f in df2.iterrows():
    filename = f['name'].split('/')[-1]
    parts = filename.split("_")
    speaker.append(parts[0])
    target.append(parts[2])

In [36]:
df2 = df2.assign(emotion = target)
df2 = df2.assign(speaker_name = speaker)

In [37]:
df2 = df2.drop('name', axis = 1)

In [38]:
df = pd.concat((df1,df2), axis=1)
df.head()

,pcm_RMSenergy_sma_max,pcm_RMSenergy_sma_min,pcm_RMSenergy_sma_range,pcm_RMSenergy_sma_maxPos,pcm_RMSenergy_sma_minPos,pcm_RMSenergy_sma_amean,pcm_RMSenergy_sma_linregc1,pcm_RMSenergy_sma_linregc2,pcm_RMSenergy_sma_linregerrQ,pcm_RMSenergy_sma_stddev,...,F0_sma_de_minPos,F0_sma_de_amean,F0_sma_de_linregc1,F0_sma_de_linregc2,F0_sma_de_linregerrQ,F0_sma_de_stddev,F0_sma_de_skewness,F0_sma_de_kurtosis,emotion,speaker_name
0,0.019556,0.000098,0.019459,28.0,0.0,0.005757,-0.000034,0.007680,0.000014,0.003969,...,72.0,-5.437617e-08,-0.009950,0.562178,239.2275,15.470450,-1.312268e-02,11.23654,anxiety,cc
1,0.020914,0.000058,0.020856,63.0,0.0,0.006449,-0.000096,0.012574,0.000024,0.006018,...,38.0,-1.862645e-08,-0.012806,0.813158,710.0782,26.651490,1.785000e-02,12.79744,anxiety,cc
2,0.027904,0.000117,0.027787,88.0,128.0,0.007473,-0.000076,0.012335,0.000039,0.006879,...,0.0,0.000000e+00,0.000000,0.000000,0.0000,0.000000,0.000000e+00,0.00000,anxiety,cc
3,0.052467,0.000205,0.052262,40.0,109.0,0.015300,-0.000016,0.016177,0.000199,0.014111,...,24.0,0.000000e+00,-0.008456,0.460871,328.6690,18.131220,2.774320e-17,20.26844,anxiety,cc
4,0.074559,0.000146,0.074414,107.0,139.0,0.016310,0.000079,0.010823,0.000423,0.020823,...,17.0,1.362392e-08,-0.005247,0.364690,67.0573,8.191598,8.360357e-01,19.55287,anxiety,cc


In [39]:
normalization_dict = {}
for speaker in df.speaker_name.unique():
    speaker_df = df.loc[df['speaker_name'] == speaker]
    for col in list(df1.columns):
        speaker_df[col] = (speaker_df[col] - speaker_df[col].mean()) / speaker_df[col].std()
    normalization_dict[speaker] = speaker_df

In [40]:
frames = []
for key, val in normalization_dict.items():
    frames.append(val)
normalized_df = pd.concat(frames)
normalized_df.head()

,pcm_RMSenergy_sma_max,pcm_RMSenergy_sma_min,pcm_RMSenergy_sma_range,pcm_RMSenergy_sma_maxPos,pcm_RMSenergy_sma_minPos,pcm_RMSenergy_sma_amean,pcm_RMSenergy_sma_linregc1,pcm_RMSenergy_sma_linregc2,pcm_RMSenergy_sma_linregerrQ,pcm_RMSenergy_sma_stddev,...,F0_sma_de_minPos,F0_sma_de_amean,F0_sma_de_linregc1,F0_sma_de_linregc2,F0_sma_de_linregerrQ,F0_sma_de_stddev,F0_sma_de_skewness,F0_sma_de_kurtosis,emotion,speaker_name
0,-0.542097,0.247067,-0.543588,-0.676082,-0.375596,-0.334244,-0.045908,-0.215893,-0.552673,-0.656059,...,0.730800,-0.261734,0.364155,-0.418834,-0.409591,-0.115231,-0.150509,0.416525,anxiety,cc
1,-0.480289,-0.262144,-0.479914,0.691267,-0.375596,-0.201482,-0.920504,0.460018,-0.487494,-0.313397,...,0.017573,-0.261734,0.307717,-0.313639,0.469244,0.733257,-0.086409,0.626721,anxiety,cc
2,-0.162182,0.497143,-0.164121,1.667944,0.664677,-0.005156,-0.633437,0.427007,-0.373351,-0.169497,...,-0.779562,-0.261734,0.560810,-0.654463,-0.856105,-1.289227,-0.123351,-1.096626,anxiety,cc
3,0.955751,1.635987,0.951074,-0.207277,0.510262,1.495457,0.205572,0.957505,0.778980,1.039349,...,-0.276108,-0.261734,0.393677,-0.461295,-0.242650,0.086685,-0.123351,1.632791,anxiety,cc
4,1.961255,0.869537,1.960431,2.410219,0.754076,1.689164,1.537263,0.218152,2.400886,2.161293,...,-0.422949,-0.261734,0.457101,-0.501608,-0.730943,-0.667596,1.606879,1.536430,anxiety,cc


In [43]:
normalized_df.head()

,pcm_RMSenergy_sma_max,pcm_RMSenergy_sma_min,pcm_RMSenergy_sma_range,pcm_RMSenergy_sma_maxPos,pcm_RMSenergy_sma_minPos,pcm_RMSenergy_sma_amean,pcm_RMSenergy_sma_linregc1,pcm_RMSenergy_sma_linregc2,pcm_RMSenergy_sma_linregerrQ,pcm_RMSenergy_sma_stddev,...,F0_sma_de_minPos,F0_sma_de_amean,F0_sma_de_linregc1,F0_sma_de_linregc2,F0_sma_de_linregerrQ,F0_sma_de_stddev,F0_sma_de_skewness,F0_sma_de_kurtosis,emotion,speaker_name
0,-0.542097,0.247067,-0.543588,-0.676082,-0.375596,-0.334244,-0.045908,-0.215893,-0.552673,-0.656059,...,0.730800,-0.261734,0.364155,-0.418834,-0.409591,-0.115231,-0.150509,0.416525,anxiety,cc
1,-0.480289,-0.262144,-0.479914,0.691267,-0.375596,-0.201482,-0.920504,0.460018,-0.487494,-0.313397,...,0.017573,-0.261734,0.307717,-0.313639,0.469244,0.733257,-0.086409,0.626721,anxiety,cc
2,-0.162182,0.497143,-0.164121,1.667944,0.664677,-0.005156,-0.633437,0.427007,-0.373351,-0.169497,...,-0.779562,-0.261734,0.560810,-0.654463,-0.856105,-1.289227,-0.123351,-1.096626,anxiety,cc
3,0.955751,1.635987,0.951074,-0.207277,0.510262,1.495457,0.205572,0.957505,0.778980,1.039349,...,-0.276108,-0.261734,0.393677,-0.461295,-0.242650,0.086685,-0.123351,1.632791,anxiety,cc
4,1.961255,0.869537,1.960431,2.410219,0.754076,1.689164,1.537263,0.218152,2.400886,2.161293,...,-0.422949,-0.261734,0.457101,-0.501608,-0.730943,-0.667596,1.606879,1.536430,anxiety,cc


In [44]:
columns = list(normalized_df.columns)

In [45]:
columns.remove('emotion')

In [46]:
columns.remove('speaker_name')

In [47]:
imputation = make_pipeline(SimpleImputer(strategy='constant'))
preprocessing = make_column_transformer((imputation, columns))

In [48]:
report = []

import xgboost as xgb

model=xgb.XGBClassifier()

for speaker in list(normalized_df.speaker_name.unique()):
    train_data = normalized_df.loc[normalized_df['speaker_name'] != speaker]
    test_data = normalized_df.loc[normalized_df['speaker_name'] == speaker]
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    data = train_data.drop(['emotion', 'speaker_name'], axis=1)
    label = train_data['emotion']
    X_processed = preprocessing.fit_transform(data)
    model.fit(X_processed, label)
    
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    test_x = test_data.drop(['emotion', 'speaker_name'], axis=1)
    test_label = test_data['emotion']
    test_x_processed = preprocessing.transform(test_x)
    y_pred = model.predict(test_x_processed)
    
    report.append(classification_report(y_true=test_label, y_pred=y_pred))

In [49]:
for model_report in report:
    print(model_report)

              precision    recall  f1-score   support

     anxiety       0.00      0.00      0.00        10
     boredom       0.00      0.00      0.00        15
  cold-anger       0.14      0.20      0.16        15
    contempt       0.25      0.45      0.32        22
     despair       0.12      0.22      0.15         9
     disgust       0.27      0.10      0.14        31
     elation       0.20      0.25      0.22        16
       happy       0.12      0.13      0.13        23
   hot-anger       0.57      0.57      0.57        14
    interest       0.08      0.06      0.07        17
     neutral       0.00      0.00      0.00        18
       panic       0.44      0.44      0.44        18
       pride       0.15      0.09      0.11        23
     sadness       0.19      0.31      0.24        13
       shame       0.36      0.19      0.25        21

    accuracy                           0.20       265
   macro avg       0.19      0.20      0.19       265
weighted avg       0.20   